In [0]:
!curl https://colab.chainer.org/install | sh -
#cupy library를 사용하기 위해 사용

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
file_data   = "/content/drive/My Drive/AssignmentFolder/Assignment10/mnist.csv"
handle_file = open(file_data, "r")
data        = handle_file.readlines()
handle_file.close()

size_row    = 28    # height of the image
size_col    = 28    # width of the image

num_image   = len(data)
count       = 0     # count for the number of images

#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = np.sum(d)
    # r = np.sqrt(s)

    return(s)

#
# make a matrix each column of which represents an images in a vector form
#
list_image  = np.empty((size_row * size_col, num_image), dtype=float)
list_label  = np.empty(num_image, dtype=int)

for line in data:

    line_data   = line.split(',')
    label       = line_data[0]
    im_vector   = np.asfarray(line_data[1:])
    im_vector   = normalize(im_vector)

    list_label[count]       = label
    list_image[:, count]    = im_vector

    count += 1

#
# plot first 150 images out of 10,000 with their labels
#
f1 = plt.figure(1)

for i in range(150):

    label       = list_label[i]
    im_vector   = list_image[:, i]
    im_matrix   = im_vector.reshape((size_row, size_col))

    plt.subplot(10, 15, i+1)
    plt.title(label)
    plt.imshow(im_matrix, cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)


#plt.show()

#
# plot the average image of all the images for each digit
#
f2 = plt.figure(2)

im_average  = np.zeros((size_row * size_col, 10), dtype=float)
im_count    = np.zeros(10, dtype=int)

for i in range(num_image):

    im_average[:, list_label[i]] += list_image[:, i]
    im_count[list_label[i]] += 1

for i in range(10):

    im_average[:, i] /= im_count[i]

    plt.subplot(2, 5, i+1)
    plt.title(i)
    plt.imshow(im_average[:,i].reshape((size_row, size_col)), cmap='Greys', interpolation='None')

    frame   = plt.gca()
    frame.axes.get_xaxis().set_visible(False)
    frame.axes.get_yaxis().set_visible(False)

plt.show()

In [0]:
#X_train : training images
train_data = list_image[0:, 0:1000]
X_train = train_data
#X_test : testing images
test_data = list_image[0:, 1000:10000]
X_test = test_data
#효율적인 값 도출을 위해 random값 고정
np.random.seed(1020)

#np.random.randn : mean = 0, deriviation = 1인 random값
u = np.random.randn(197, 785)
v = np.random.randn(50, 197)
w = np.random.randn(10, 50)
#train
#(784, 1)
X = []
#(196, 1)
Y = []
#(49, 1)
Z = []
#(10, 1)
H = []
#test
#(784,1)
x = []
#(196,1)
y = []
#(49,1)
z = []
#(10,1)
h = []
#bias
bias = 1
#계속 사용하는 function


In [0]:
#구해야 하는 train label
train_label = list_label[0:1000]
real_label = np.zeros((10, 1000))
#label
for i in range(0, 1000):
  for j in range(0, 10):
    if j == train_label[i]:
      real_label[j][i] = 1
    else:
      real_label[j][i] = 0
      
#구해야 하는 test label
test_label = list_label[1000:10000]
test_real_label = np.zeros((10, 9000))
for i in range(0, 9000):
  for j in range(0, 10):
    if j == test_label[i]:
      test_real_label[j][i] = 1
    else:
      test_real_label[j][i] = 0

In [0]:
#use mini batch 
#상수
iter = 0
learning_rate = 0.1

train_loss_list = []
test_loss_list = []

#accuracy
train_accuracy_list = []
test_accuracy_list = []
train_correct = 0
test_correct = 0

threshold = 0.000005
#correct / miss list
correct_label = []
correct_image = []
miss_label = []
miss_image = []
#to use convergence append 0
train_loss_list.append(0)

def sigmoid (z):
    return 1 / (1 + np.exp(-z))
 
def sig_der(z):
    return (z*(1-z))

while iter < 30:
  for i in range(0, 9000):
    #test

    x = X_test[0:, i: (i+1)]
    x = np.vstack([x, bias])
    y = sigmoid(cp.dot(u, x))
    z = sigmoid(cp.dot(v, y))
    h = sigmoid(cp.dot(w, z))
      
    r = test_real_label[0:, i: (i + 1)]
    h_loss = h - r

    if test_real_label[h.argmax()][i] == 1:
        
      test_correct += 1

    #training input
    if i < 1000:
      #mini_batch
      X = X_train[0:, i:i+1]
      X = np.vstack([X, bias])
      Y = sigmoid(cp.dot(u, X))
      Z = sigmoid(cp.dot(v, Y))
      H = sigmoid(cp.dot(w, Z))

      R = real_label[0:,i:i+1]
      #-값을 가지기도 해서 graph상에 표시할 때는 abs값 적용
      H_loss = H - R
      #to use backpropagation
      #bias는 어차피 1이므로 신경쓰지 않아도 된다
      H_der = H_loss * sig_der(H)
      Z_loss = cp.dot(w.T, H_der)
      Z_der = Z_loss * sig_der(Z)
      Y_loss = cp.dot(v.T, Z_der)
      Y_der = Y_loss * sig_der(Y)

      tmpw = w - learning_rate * Z.T * H_der
      tmpv = v - learning_rate * Y.T * Z_der
      tmpu = u - learning_rate * X.T * Y_der
      w = tmpw
      v = tmpv
      u = tmpu
      if real_label[H.argmax()][i] == 1:
        train_correct += 1
        correct_label.append(H.argmax())
        correct_image.append(X_train[0:, i:i+1])
      else:
        miss_label.append(H.argmax())
        miss_image.append(X_train[0:, i:i+1])
    #End batch
  #loss check
  train_loss_list.append(np.mean(np.abs(H_loss)))
  test_loss_list.append(np.mean(np.abs(h_loss)))
  #accuracy check
  train_accuracy_list.append(train_correct / 1000)
  test_accuracy_list.append(test_correct / 9000)
  #converge check
  if abs(train_loss_list[iter] - train_loss_list[iter + 1]) < threshold:
    print("iter : ", iter)
    print("converge")
    break

  train_correct = 0
  test_correct = 0
  iter += 1

#delete 0
del train_loss_list[0]

In [0]:
plt.plot(train_loss_list)
plt.show()

In [0]:
print(train_accuracy_list[len(train_accuracy_list) - 1])

In [0]:
plt.plot(train_accuracy_list)
plt.show()

In [0]:
plt.plot(test_loss_list)
plt.show()

In [0]:
print(test_accuracy_list[len(test_accuracy_list) - 1])

In [0]:
plt.plot(test_accuracy_list)
plt.show()

In [0]:
for i in range(10):
  plt.subplot(2, 5,i+1)
  plt.title(correct_label[i+10000])
  plt.imshow(correct_image[i+10000].reshape(28,28, order = 'C'), cmap='Greys', interpolation='None')
  frame = plt.gca()
  frame.axes.get_xaxis().set_visible(False)
  frame.axes.get_yaxis().set_visible(False)
plt.show()

In [0]:
for i in range(10):
  plt.subplot(2,5,i+1)
  plt.title(miss_label[i + 10000])
  plt.imshow(miss_image[i + 10000].reshape(28,28, order = 'C'), cmap='Greys', interpolation='None')
  frame = plt.gca()
  frame.axes.get_xaxis().set_visible(False)
  frame.axes.get_yaxis().set_visible(False)
plt.show()